### Movies

#### Se creo una base de datos con una lista de peliculas, en la interfaz gráfica podremos seleccionar un género de pelicula y el país de producción y nos mostrará la lista de peliculas correspondientes a los filtros seleccionados.

In [1]:
import tkinter as tk
from tkinter import ttk
import pyodbc

In [2]:
# Información de la base de datos

server = 'JESS\SQLEXPRESS'
bd = 'Movies'

In [3]:
# Función para obtener los géneros de las peliculas

def opciones_genre():
    try:
        conn = pyodbc.connect(driver='{SQL server}', host = server, database = bd)
        cursor = conn.cursor()

        # Se obtienen los generos unicos de la tabla "Movies"
        cursor.execute("SELECT DISTINCT genre FROM Movies")
        generos = [row[0].strip() for row in cursor.fetchall()]

        conn.close()
        return generos
        
    except Exception as e:
        print(f"Error al obtener géneros: {e}")
        return []

In [4]:
# Función para obtener los países

def obtener_country(genre):
    try:
        conn = pyodbc.connect(driver='{SQL server}', host = server, database = bd)
        cursor = conn.cursor()

        # Se obtienen los paises unicos de la tabla "Movies" dependiendo del filtro "genero".
        query = "SELECT DISTINCT country FROM Movies WHERE genre = ?"
        cursor.execute(query, (genre,))
        paises = [row[0].strip() for row in cursor.fetchall()]

        conn.close()
        return paises
        
    except Exception as e:
        print(f"Error al obtener países: {e}")
        return []

In [5]:
# Función para obtener las películas filtradas por género y país

def obtener_peliculas(genre, country):
    try:
        conn = pyodbc.connect(driver='{SQL server}', host = server, database = bd)
        cursor = conn.cursor()

        # Consulta SQL para obtener las películas filtradas por género y país
        query = "SELECT * FROM Movies WHERE genre = ? AND country = ?"
        cursor.execute(query, (genre, country))
        peliculas = cursor.fetchall()

        conn.close()
        return peliculas
        
    except Exception as e:
        print(f"Error al obtener películas: {e}")
        return []

In [6]:
# Función para mostrar las películas filtradas

def mostrar_peliculas():
    # Obtener los valores de los filtros
    genre = gen_combobox.get()
    country = pais_combobox.get()

    # Obtener las películas filtradas
    peliculas = obtener_peliculas(genre, country)

    # Limpiar la lista de películas mostradas
    for item in lista_peliculas.get_children():
        lista_peliculas.delete(item)

    # Mostrar las películas
    for pelicula in peliculas:
        lista_peliculas.insert("", "end", values=(
            pelicula[0].strip(), 
            pelicula[1].strip(), 
            str(pelicula[2]).strip(),
            pelicula[3].strip(),
            pelicula[4].strip(),
            pelicula[5].strip()
        ))

In [7]:
# Función para actualizar el filtro de paises dependiendo de lo que se seleccione en el primer filtro

def actualizar_paises(event):
    genre = gen_combobox.get() 
    if genre:
        paises = obtener_country(genre) 
        pais_combobox['values'] = paises
        pais_combobox.set('')

In [9]:
# Creamos la ventana

root = tk.Tk()
root.title("Movies")

# Mostramos la aplicacion al frente de otras
root.attributes("-topmost", True)

# Cambiamos el color del background
root.config(bg="white")

#Asignamos un logotipo
root.iconbitmap('pelicula.ico')

# Obtener los géneros desde la tabla
generos = opciones_genre()

# Crear los menús desplegables
gen_label = tk.Label(root, width=30, bd=1, relief="solid", font=("Arial", 11), text="Género:")
gen_label.grid(row=0, column=0, padx=10, pady=10)
gen_combobox = ttk.Combobox(root, width=30, font=("Arial", 10), values=generos)
gen_combobox.set(generos[0] if generos else '')
gen_combobox.grid(row=0, column=1, padx=10, pady=10)

pais_label = tk.Label(root, width=30, bd=1, relief="solid", font=("Arial", 11), text="País:")
pais_label.grid(row=1, column=0, padx=10, pady=10)
pais_combobox = ttk.Combobox(root, width=30, font=("Arial", 10))
pais_combobox.grid(row=1, column=1, padx=10, pady=10)

gen_combobox.bind("<<ComboboxSelected>>", actualizar_paises)

# Botón para mostrar las películas
buscar_button = tk.Button(root, bg="lightblue", width=10, height=1, font=("Arial", 11), text="Buscar", command=mostrar_peliculas)
buscar_button.grid(row=0, column=2, padx=10, pady=10)

# Crear un Treeview para mostrar las películas
lista_peliculas = ttk.Treeview(root, columns=("Name", "Genre", "Year", "Director", "Writer", "Country"), show="headings")
lista_peliculas.heading("Name", text="Name")
lista_peliculas.heading("Genre", text="Genre")
lista_peliculas.heading("Year", text="Year")
lista_peliculas.heading("Director", text="Director")
lista_peliculas.heading("Writer", text="Writer")
lista_peliculas.heading("Country", text="Country")

lista_peliculas.grid(row=2, column=0, columnspan=10, padx=10, pady=10)

# Ejecutar la ventana
root.mainloop()
